In [1]:
from spai.storage import Storage

# retrieve all images
storage = Storage("../../data")
images = storage.list("NDVI*")
print("Found the following images:", images)

Found the following images: []


In [2]:
import numpy as np

# categorize ndvis

def categorize(data, cats=[-100, 0.1, 0.3, 0.5, 100]):
    data0 = data.copy()
    for i, cat in enumerate(cats[:-1]):
        data0[(data > cat) & (data <= cats[i+1])] = i
    data0 = data0.astype(np.uint8)
    return data0
       
for image in images:
    ds = storage.read(image)
    data = ds.read(1)
    cats = categorize(data)
    date = image.split("_")[1].split(".")[0]
    storage.create(cats, f"cat_NDVI_{date}.tif", ds=ds)

In [3]:
import matplotlib.pyplot as plt

# visualize categorized ndvis

images = storage.list("cat_NDVI*")
for image in images:
    ds = storage.read(image)
    cats = ds.read(1)
    date = image.split("_")[2].split(".")[0]
    plt.imshow(cats)
    plt.title(date)
    plt.axis("off")
    plt.show()


In [4]:
import pandas as pd

# compute statistics

stats = []
mppx = 10 * 10  # meters per pixel
hasppx = mppx / 10000  # hectares per square meter
for image in images:
    ds = storage.read(image)
    data = ds.read(1)
    counts = np.bincount(data.flatten())
    has = counts * hasppx
    stats.append(has)

dates = [image.split("_")[2].split(".")[0] for image in images]
df = pd.DataFrame(stats, columns=['bare ground', 'low', 'medium', 'high'], index=dates)
df.index = pd.to_datetime(df.index)
df = df.sort_index()
df

,bare ground,low,medium,high


In [1]:
from spai.project import ProjectConfig

p = ProjectConfig()
p

In [2]:
p.aoi

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[3.07688064333496, 42.34062833138711],
      [3.174751318648111, 42.34062833138711],
      [3.174751318648111, 42.39784193720783],
      [3.07688064333496, 42.39784193720783],
      [3.07688064333496, 42.34062833138711]]]}}]}

In [5]:
p.dates

['2020-08-01', '2020-08-21']

In [6]:
import geopandas as gpd

url = 'https://data.source.coop/cholmes/google-open-buildings/geoparquet-by-country/country_iso=AF/AF.parquet'
gdf = gpd.read_parquet(url)
gdf

ArrowInvalid: Expected a local filesystem path, got a URI: 'https://data.source.coop/cholmes/google-open-buildings/geoparquet-by-country/country_iso=AF/AF.parquet'